In [20]:
!pip install mtcnn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
# -*- coding: utf-8 -*-
import cv2
import os
from os import listdir
from PIL import Image
from numpy import asarray
import matplotlib.pyplot as plt 
from mtcnn import MTCNN
import pickle
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image 
import os
from tensorflow.keras.utils import image_dataset_from_directory, to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input
print(tf.__version__)

2.8.2


In [22]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [23]:
path = "/content/drive/MyDrive/Face Recognition"
os.chdir(path)
!ls

Dataset  Face_recognition.ipynb  faces-embeddings.npz  Model


In [24]:
videopath = '/content/drive/MyDrive/Face Recognition'

In [25]:
listvideo = os.listdir(videopath+'/Dataset/Video')

In [26]:
listvideo

['Tuấn.mp4',
 'Trường.mp4',
 'Linh.mp4',
 'Hiếu.mp4',
 'Quân.mp4',
 'Xuân Anh.mp4',
 'Tân.mp4',
 'Đức.mp4',
 'Kiên.mp4',
 'Việt Đức.mp4',
 'Hùng.mp4',
 'Thắng.mp4',
 'Vân.mp4',
 'HĐức.mp4']

In [27]:
def extract_face(filename, required_size=(160, 160)):
    image = Image.open(filename)
    image = image.convert('RGB')
    pixels = asarray(image)
    detector = MTCNN()
    results = detector.detect_faces(pixels)
    x1, y1, width, height = results[0]['box']
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    face = pixels[y1:y2, x1:x2]
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = asarray(image)
    return face_array
def export_image(path):
    videopath = path + "/Dataset/Video"
    listvideo = os.listdir(videopath)
    for a in listvideo:
        n = a[:len(a)-4]
        datapath1 = path + "/Dataset"+ "/raw/"+a[:len(a)-4]
        if os.path.exists(datapath1) != True:
            os.mkdir(datapath1)
            datapath2 = path + "/Dataset"+ "/process/" +a[:len(a)-4]
            if os.path.exists(datapath2) != True:
                os.mkdir(datapath2)
            apath = path + "/Dataset/Video" + "/" + a
            avideo = cv2.VideoCapture(apath)
            status, image = avideo.read()
            status = True
            count = 0
            while count<30:
                bpath = path + "/Dataset/raw/" + n +"/"+n +str(count) + ".jpg"
                os.chdir(path+'/Dataset/raw/'+n)
                cv2.imwrite(n+"%d.jpg"%count, image)
                os.chdir(path+'/Dataset/process/'+n)
                cv2.imwrite(n+"%d.jpg"%count,cv2.cvtColor(extract_face(bpath), cv2.COLOR_BGR2RGB))
                status, image = avideo.read()
                count += 1
            print("completed {}".format(n))

In [28]:
export_image(path)

In [29]:
listdir = os.listdir(path + '/Dataset/process')

print(listdir)

['Tuấn', 'Trường', 'Linh', 'Hiếu', 'Quân', 'Xuân Anh', 'Tân', 'Đức', 'Kiên', 'Việt Đức', 'Hùng', 'Thắng', 'Vân', 'HĐức']


In [30]:
x = []
y = []

path1 = path + '/Dataset/process'

i = 0
for folder in os.listdir(path1):
    a = 0
    print(folder +': ' + str(i))
    for filename in os.listdir(os.path.join(path1, folder)):
        y.append(folder)
        x.append(np.asarray(Image.open(path1 + '/' + folder + '/' + filename)))
        a += 1
    print('Added {} images of {}'.format(a, folder))
    print('')
    i += 1

Tuấn: 0
Added 30 images of Tuấn

Trường: 1
Added 30 images of Trường

Linh: 2
Added 30 images of Linh

Hiếu: 3
Added 30 images of Hiếu

Quân: 4
Added 30 images of Quân

Xuân Anh: 5
Added 30 images of Xuân Anh

Tân: 6
Added 30 images of Tân

Đức: 7
Added 30 images of Đức

Kiên: 8
Added 25 images of Kiên

Việt Đức: 9
Added 30 images of Việt Đức

Hùng: 10
Added 30 images of Hùng

Thắng: 11
Added 30 images of Thắng

Vân: 12
Added 30 images of Vân

HĐức: 13
Added 30 images of HĐức



In [31]:
x = np.array(x)
y = np.array(y)
print(x.shape)
print(y.shape)

(415, 160, 160, 3)
(415,)


In [32]:
sss = StratifiedShuffleSplit(n_splits=3, test_size=0.2, random_state=42)

for train_index, val_index in sss.split(x, y):
    x_train, x_val = x[train_index], x[val_index]
    y_train, y_val = y[train_index], y[val_index]

In [42]:
path = "/content/drive/MyDrive/Face Recognition/Model"
os.chdir(path)
!ls

Model.h5


In [41]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [43]:
from numpy import load
from numpy import expand_dims
from keras.models import load_model
from os import listdir
from os.path import isdir
from PIL import Image
from matplotlib import pyplot
from numpy import savez_compressed
from numpy import asarray
from mtcnn.mtcnn import MTCNN

def get_embedding(model, face_pixels):
    face_pixels = face_pixels.astype('float32')
    mean, std = face_pixels.mean(), face_pixels.std()
    face_pixels = (face_pixels - mean) / std
    samples = expand_dims(face_pixels, axis=0)
    yhat = model.predict(samples)
    return yhat[0]

print('Loaded: ', x_train.shape, y_train.shape, x_val.shape, y_val.shape)
model = load_model('Model.h5')  
print('Loaded Model')
newTrainX = list()
for face_pixels in x_train:
    embedding = get_embedding(model, face_pixels)
    newTrainX.append(embedding)
newTrainX = asarray(newTrainX)
print(newTrainX.shape)
newvalX = list()
for face_pixels in x_val:
    embedding = get_embedding(model, face_pixels)
    newvalX.append(embedding)
newvalX = asarray(newvalX)
print(newvalX.shape)
savez_compressed('faces-embeddings.npz', newTrainX, y_train, newvalX, y_val)

Loaded:  (332, 160, 160, 3) (332,) (83, 160, 160, 3) (83,)


Loaded Model
(332, 128)
(83, 128)


In [44]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
from random import choice
from sklearn.svm import SVC
from matplotlib import pyplot

data = load('faces-embeddings.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
print('Dataset: train=%d, test=%d' % (trainX.shape[0], testX.shape[0]))

in_encoder = Normalizer(norm='l2')
trainX = in_encoder.transform(trainX)
testX = in_encoder.transform(testX)

out_encoder = LabelEncoder()
out_encoder.fit(trainy)
trainy = out_encoder.transform(trainy)
testy = out_encoder.transform(testy)

model = SVC(kernel='linear', probability=True)
model.fit(trainX, trainy)

yhat_train = model.predict(trainX)
yhat_test = model.predict(testX)
# score
score_train = accuracy_score(trainy, yhat_train)
score_test = accuracy_score(testy, yhat_test)
# summarize
print('Accuracy: train=%.3f, test=%.3f' % (score_train*100, score_test*100))

testX_faces = data['arr_2']


Dataset: train=332, test=83
Accuracy: train=100.000, test=100.000


In [36]:
selection = 5
random_face_pixels = testX_faces[selection]
random_face_emb = testX[selection]
random_face_class = testy[selection]
random_face_name = out_encoder.inverse_transform([random_face_class])

samples = expand_dims(random_face_emb, axis=0)
yhat_class = model.predict(samples)
yhat_prob = model.predict_proba(samples)

class_index = yhat_class[0]
class_probability = yhat_prob[0,class_index] * 100
predict_names = out_encoder.inverse_transform(yhat_class)
print('Predicted: %s (%.3f)' % (predict_names[0], class_probability))
print('Expected: %s' % random_face_name[0])

Predicted: Hiếu (77.982)
Expected: Hiếu


In [75]:
path = '/content/drive/MyDrive/Face Recognition'

In [76]:
videopath = path + "/Dataset/Video"
listvideo = os.listdir(videopath)

In [77]:
listvideo

['Tuấn.mp4',
 'Trường.mp4',
 'Linh.mp4',
 'Hiếu.mp4',
 'Quân.mp4',
 'Xuân Anh.mp4',
 'Tân.mp4',
 'Đức.mp4',
 'Kiên.mp4',
 'Việt Đức.mp4',
 'Hùng.mp4',
 'Thắng.mp4',
 'Vân.mp4',
 'HĐức.mp4']

In [81]:
def export_image1(path):
    videopath = path + "/Dataset/Video"
    listvideo = os.listdir(videopath)
    for a in listvideo:
        n = a[:len(a)-4]
        datapath = path + "/Dataset"+ "/Test"

        apath = path + "/Dataset/Video" + "/" + a
        avideo = cv2.VideoCapture(apath)
        status, image = avideo.read()
        status = True
        count = 0
        while count<10:
            bpath = path + "/Dataset/Test/" + n +"/"+n +str(count) + ".jpg"
            os.chdir(path+'/Dataset/Test')
            cv2.imwrite(n+"%d.jpg"%count, image)
            status, image = avideo.read()
            count += 1
        print("completed {}".format(n))

In [ ]:
export_image1(path)

In [85]:
#take input an image, find it's face embedding and then get prediction:
from os import listdir
from os.path import isdir

test_img = list()
list_item = list()
path = "/content/drive/MyDrive/Face Recognition/Dataset/Test"
for item in os.listdir(path):
  list_item.append(item)
  path_img = path+'/'+item
  face = extract_face(path_img)
  test_img.append(face)
                      

In [87]:
len_test = len(test_img)

In [88]:
len_test

140

In [89]:
path = "/content/drive/MyDrive/Face Recognition/Model"
os.chdir(path)
!ls

faces-embeddings.npz  Model.h5


In [90]:
from keras.models import load_model
model_nn = load_model('Model.h5')

In [91]:
from numpy import expand_dims
test_emb = list()
for i in range(len_test):
  t_emb = get_embedding(model_nn, test_img[i])
  test_emb.append(t_emb)

In [92]:
for i in range(len_test):
  print(list_item[i])
  samples = expand_dims(test_emb[i], axis=0)
  print(samples.shape)
  yhat_class = model.predict(samples)
  yhat_prob = model.predict_proba(samples)
  class_index = yhat_class[0]
  class_probability = yhat_prob[0,class_index] * 100
  predict_names = out_encoder.inverse_transform(yhat_class)
  print('Predicted: %s (%.3f)' % (predict_names[0], class_probability))

Tuấn0.jpg
(1, 128)
Predicted: Tuấn (100.000)
Tuấn1.jpg
(1, 128)
Predicted: Tuấn (100.000)
Tuấn2.jpg
(1, 128)
Predicted: Tuấn (100.000)
Tuấn3.jpg
(1, 128)
Predicted: Tuấn (100.000)
Tuấn4.jpg
(1, 128)
Predicted: Tuấn (100.000)
Tuấn5.jpg
(1, 128)
Predicted: Tuấn (100.000)
Tuấn6.jpg
(1, 128)
Predicted: Tuấn (100.000)
Tuấn7.jpg
(1, 128)
Predicted: Tuấn (100.000)
Tuấn8.jpg
(1, 128)
Predicted: Tuấn (100.000)
Tuấn9.jpg
(1, 128)
Predicted: Tuấn (100.000)
Trường0.jpg
(1, 128)
Predicted: Trường (100.000)
Trường1.jpg
(1, 128)
Predicted: Trường (100.000)
Trường2.jpg
(1, 128)
Predicted: Trường (100.000)
Trường3.jpg
(1, 128)
Predicted: Trường (100.000)
Trường4.jpg
(1, 128)
Predicted: Trường (100.000)
Trường5.jpg
(1, 128)
Predicted: Trường (100.000)
Trường6.jpg
(1, 128)
Predicted: Trường (100.000)
Trường7.jpg
(1, 128)
Predicted: Trường (100.000)
Trường8.jpg
(1, 128)
Predicted: Trường (100.000)
Trường9.jpg
